Copyright (c) Microsoft Corporation. All rights reserved.  
Licensed under the MIT License.

![Impressions](https://PixelServer20190423114238.azurewebsites.net/api/impressions/MachineLearningNotebooks/tutorials/machine-learning-pipelines-advanced/tutorial-pipeline-batch-scoring-classification.png)

# Use Azure Machine Learning Pipelines for batch prediction
In this tutorial, you use Azure Machine Learning service pipelines to run a batch scoring image classification job. The example job uses the pre-trained [Inception-V3](https://arxiv.org/abs/1512.00567) CNN (convolutional neural network) Tensorflow model to classify unlabeled images. Machine learning pipelines optimize your workflow with speed, portability, and reuse so you can focus on your expertise, machine learning, rather than on infrastructure and automation. After building and publishing a pipeline, you can configure a REST endpoint to enable triggering the pipeline from any HTTP library on any platform.

In this tutorial, you learn the following tasks:

> * Configure workspace and download sample data
> * Create data objects to fetch and output data
> * Download, prepare, and register the model to your workspace
> * Provision compute targets and create a scoring script
> * Use ParallelRunStep to do batch scoring
> * Build, run, and publish a pipeline
> * Enable a REST endpoint for the pipeline

If you don't have an Azure subscription, create a free account before you begin. Try the [free or paid version of Azure Machine Learning service](https://aka.ms/AMLFree) today.

## Prerequisites

* Complete the [setup tutorial](https://docs.microsoft.com/azure/machine-learning/service/tutorial-1st-experiment-sdk-setup) if you don't already have an Azure Machine Learning service workspace or notebook virtual machine.
* After you complete the setup tutorial, open the **tutorials/tutorial-pipeline-batch-scoring-classification.ipynb** notebook using the same notebook server.

This tutorial is also available on [GitHub](https://github.com/Azure/MachineLearningNotebooks/tree/master/tutorials) if you wish to run it in your own [local environment](how-to-configure-environment.md#local). Run `pip install azureml-sdk[notebooks] azureml-pipeline-core azureml-pipeline-steps pandas requests` to get the required packages.

### Install required libraries

Run the following cell to install required libraries.

In [0]:
pip install azureml-sdk[notebooks] azureml-pipeline-core azureml-pipeline-steps pandas requests

Python interpreter will be restarted.
Collecting azureml-sdk[notebooks]
 Downloading azureml_sdk-1.17.0-py3-none-any.whl (4.4 kB)
Collecting azureml-pipeline-core
 Downloading azureml_pipeline_core-1.17.0-py3-none-any.whl (303 kB)
Collecting azureml-pipeline-steps
 Downloading azureml_pipeline_steps-1.17.0-py3-none-any.whl (60 kB)
Requirement already satisfied: pandas in /databricks/python3/lib/python3.7/site-packages (1.0.1)
Requirement already satisfied: requests in /databricks/python3/lib/python3.7/site-packages (2.22.0)
Collecting azureml-train~=1.17.0
 Downloading azureml_train-1.17.0-py3-none-any.whl (3.3 kB)
Collecting azureml-pipeline~=1.17.0
 Downloading azureml_pipeline-1.17.0-py3-none-any.whl (3.7 kB)
Collecting azureml-dataset-runtime[fuse]~=1.17.0
 Downloading azureml_dataset_runtime-1.17.0-py3-none-any.whl (3.4 kB)
Collecting azureml-train-automl-client~=1.17.0
 Downloading azureml_train_automl_client-1.17.0-py3-none-any.whl (108 kB)
Collecting azureml-core~=1.17.0
 Downloading azureml_core-1.17.0-py3-none-any.whl (2.1 MB)
Collecting azureml-widgets~=1.17.0; extra == "notebooks"
 Downloading azureml_widgets-1.17.0-py3-none-any.whl (14.1 MB)
Collecting azureml-contrib-notebook~=1.17.0; extra == "notebooks"
 Downloading azureml_contrib_notebook-1.17.0-py3-none-any.whl (29 kB)
Collecting azureml-train-core~=1.17.0
 Downloading azureml_train_core-1.17.0-py3-none-any.whl (8.6 MB)
Requirement already satisfied: python-dateutil>=2.6.1 in /databricks/python3/lib/python3.7/site-packages (from pandas) (2.8.1)
Requirement already satisfied: numpy>=1.13.3 in /databricks/python3/lib/python3.7/site-packages (from pandas) (1.18.1)
Requirement already satisfied: pytz>=2017.2 in /databricks/python3/lib/python3.7/site-packages (from pandas) (2019.3)
Requirement already satisfied: certifi>=2017.4.17 in /databricks/python3/lib/python3.7/site-packages (from requests) (2020.6.20)
Requirement already satisfied: idna<2.9,>=2.5 in /databricks/python3/lib/python3.7/site-packages (from requests) (2.8)
Requirement already satisfied: urllib3!=1.25.0,!=1.25.1,<1.26,>=1.21.1 in /databricks/python3/lib/python3.7/site-packages (from requests) (1.25.8)
Requirement already satisfied: chardet<3.1.0,>=3.0.2 in /usr/lib/python3/dist-packages (from requests) (3.0.4)
Requirement already satisfied: pyarrow<2.0.0,>=0.17.0 in /databricks/python3/lib/python3.7/site-packages (from azureml-dataset-runtime[fuse]~=1.17.0->azureml-sdk[notebooks]) (1.0.1)
Collecting azureml-dataprep<2.5.0a,>=2.4.0a
 Downloading azureml_dataprep-2.4.2-py3-none-any.whl (28.2 MB)
Collecting fusepy<4.0.0,>=3.0.1; extra == "fuse"
 Downloading fusepy-3.0.1.tar.gz (11 kB)
Collecting azureml-telemetry~=1.17.0
 Downloading azureml_telemetry-1.17.0-py3-none-any.whl (30 kB)
Collecting azureml-automl-core~=1.17.0
 Downloading azureml_automl_core-1.17.0-py3-none-any.whl (181 kB)
Collecting azure-mgmt-resource<15.0.0,>=1.2.1
 Downloading azure_mgmt_resource-10.3.0-py2.py3-none-any.whl (1.0 MB)
Collecting jsonpickle
 Downloading jsonpickle-1.4.1-py2.py3-none-any.whl (36 kB)
Collecting azure-mgmt-authorization<1.0.0,>=0.40.0
 Downloading azure_mgmt_authorization-0.61.0-py2.py3-none-any.whl (94 kB)
Collecting backports.tempfile
 Downloading backports.tempfile-1.0-py2.py3-none-any.whl (4.4 kB)
Collecting contextlib2
 Downloading contextlib2-0.6.0.post1-py2.py3-none-any.whl (9.8 kB)
Collecting azure-mgmt-containerregistry>=2.0.0
 Downloading azure_mgmt_containerregistry-2.8.0-py2.py3-none-any.whl (718 kB)
Collecting azure-graphrbac<1.0.0,>=0.40.0
 Downloading azure_graphrbac-0.61.1-py2.py3-none-any.whl (141 kB)
Collecting PyJWT
 Downloading PyJWT-1.7.1-py2.py3-none-any.whl (18 kB)
Collecting ndg-httpsclient
 Downloading ndg_httpsclient-0.5.1-py3-none-any.whl (34 kB)
Collecting azure-mgmt-storage<16.0.0,>=1.5.0
 Downloading azure_mgmt_storage-11.2.0-py2.py3-none-any.whl (547 kB)
Collecting ruamel.yaml>=0.15.35
 Downloading ruamel.yaml-0.16.12-py2.py3-none-any.whl (111 kB)
Collecting azure-mgmt-keyv

## Configure workspace and create datastore

Create a workspace object from the existing workspace. A [Workspace](https://docs.microsoft.com/python/api/azureml-core/azureml.core.workspace.workspace?view=azure-ml-py) is a class that accepts your Azure subscription and resource information. It also creates a cloud resource to monitor and track your model runs. `Workspace.from_config()` reads the file **config.json** and loads the authentication details into an object named `ws`. `ws` is used throughout the rest of the code in this tutorial.

In [0]:
#Provide the Subscription ID of your existing Azure subscription
subscription_id = "d30d76c0-804c-4f4e-ab7b-509d06a49550" # <- needs to be the subscription with the MCW-MLOps resource group

#Provide values for the existing Resource Group 
resource_group = "vlk-workshop" # <- replace XXXXX with your unique identifier

#Provide the Workspace Name and Azure Region of the Azure Machine Learning Workspace
workspace_name = "vlk-workspace" # <- replace XXXXX with your unique identifier
workspace_region = "westeurope" # <- region of your Quick-Starts resource group

In [0]:
import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.core.model import Model
from azureml.core.dataset import Dataset
from azureml.core.compute import ComputeTarget, AmlCompute, DatabricksCompute
from azureml.core.compute_target import ComputeTargetException
from azureml.widgets import RunDetails


# Check core SDK version number
print("SDK version:", azureml.core.VERSION)

SDK version: 1.17.0

In [0]:
# By using the exist_ok param, if the worskpace already exists you get a reference to the existing workspace
# allowing you to re-run this cell multiple times as desired (which is fairly common in notebooks).
ws = Workspace.create(
    name = workspace_name,
    subscription_id = subscription_id,
    resource_group = resource_group, 
    location = workspace_region,
    exist_ok = True)

ws.write_config()
print('Workspace configuration succeeded')

Performing interactive authentication. Please follow the instructions on the terminal.
To sign in, use a web browser to open the page https://microsoft.com/devicelogin and enter the code DXGTFEBRT to authenticate.
You have logged in. Now let us find all the subscriptions to which you have access...
Interactive authentication successfully completed.
Workspace configuration succeeded

### Create AML Compute Cluster
Now you are ready to create the CPU compute cluster. Run the following cell to create a new compute cluster (or retrieve the existing cluster if it already exists). The code below will create a *CPU based* cluster where each node in the cluster is of the size `Standard_D2_V2`, and the cluster is restricted to use 1 node. This will take couple of minutes to create.

In [0]:
### Create AML CPU based Compute Cluster

cluster_name = "vlk-compute"

try:
    compute_target = ComputeTarget(workspace=ws, name=cluster_name)
    print('Found existing compute target.')
except ComputeTargetException:
    print('Creating a new compute target...')
    compute_config = AmlCompute.provisioning_configuration(vm_size='Standard_D2_V2',
                                                           min_nodes=1, max_nodes=1)

    # create the cluster
    compute_target = ComputeTarget.create(ws, cluster_name, compute_config)

    compute_target.wait_for_completion(show_output=True)

# Use the 'status' property to get a detailed status for the current AmlCompute. 
print(compute_target.status.serialize())

Found existing compute target.
{'currentNodeCount': 1, 'targetNodeCount': 1, 'nodeStateCounts': {'preparingNodeCount': 0, 'runningNodeCount': 0, 'idleNodeCount': 1, 'unusableNodeCount': 0, 'leavingNodeCount': 0, 'preemptedNodeCount': 0}, 'allocationState': 'Steady', 'allocationStateTransitionTime': '2020-11-02T10:06:09.094000+00:00', 'errors': None, 'creationTime': '2020-11-02T10:03:48.827921+00:00', 'modifiedTime': '2020-11-02T10:04:04.446973+00:00', 'provisioningState': 'Succeeded', 'provisioningStateTransitionTime': None, 'scaleSettings': {'minNodeCount': 1, 'maxNodeCount': 1, 'nodeIdleTimeBeforeScaleDown': ''}, 'vmPriority': 'Dedicated', 'vmSize': 'STANDARD_D2_V2'}

### Create a datastore for sample images

Get the ImageNet evaluation public data sample from the public blob container `sampledata` on the account `pipelinedata`. Calling `register_azure_blob_container()` makes the data available to the workspace under the name `images_datastore`. Then specify the workspace default datastore as the output datastore, which you use for scoring output in the pipeline.

In [0]:
from azureml.core.datastore import Datastore

batchscore_blob = Datastore.register_azure_blob_container(ws, 
                      datastore_name="images_datastore", 
                      container_name="sampledata", 
                      account_name="pipelinedata", 
                      overwrite=True)

def_data_store = ws.get_default_datastore()

## Create data objects

When building pipelines, `Dataset` objects are used for reading data from workspace datastores, and `PipelineData` objects are used for transferring intermediate data between pipeline steps.

This batch scoring example only uses one pipeline step, but in use-cases with multiple steps, the typical flow will include:

1. Using `Dataset` objects as **inputs** to fetch raw data, performing some transformations, then **outputting** a `PipelineData` object.
1. Use the previous step's `PipelineData` **output object** as an *input object*, repeated for subsequent steps.

For this scenario you create `Dataset` objects corresponding to the datastore directories for both the input images and the classification labels (y-test values). You also create a `PipelineData` object for the batch scoring output data.

In [0]:
from azureml.core.dataset import Dataset
from azureml.pipeline.core import PipelineData

input_images = Dataset.File.from_files((batchscore_blob, "batchscoring/images/"))
label_ds = Dataset.File.from_files((batchscore_blob, "batchscoring/labels/"))
output_dir = PipelineData(name="scores", datastore=def_data_store)

Next, we need to register the datasets with the workspace.

In [0]:
input_images = input_images.register(workspace=ws, name="input_images")
label_ds = label_ds.register(workspace=ws, name="label_ds", create_new_version=True)

## Download and register the model

Download the pre-trained Tensorflow model to use it for batch scoring in the pipeline. First create a local directory where you store the model, then download and extract it.

In [0]:
import os
import tarfile
import urllib.request

if not os.path.isdir("models"):
    os.mkdir("models")
    
response = urllib.request.urlretrieve("http://download.tensorflow.org/models/inception_v3_2016_08_28.tar.gz", "model.tar.gz")
tar = tarfile.open("model.tar.gz", "r:gz")
tar.extractall("models")

Now you register the model to your workspace, which allows you to easily retrieve it in the pipeline process. In the `register()` static function, the `model_name` parameter is the key you use to locate your model throughout the SDK.

In [0]:
import shutil
from azureml.core.model import Model

# register downloaded model 
model = Model.register(model_path="models/inception_v3.ckpt",
                       model_name="inception",
                       tags={"pretrained": "inception"},
                       description="Imagenet trained tensorflow inception",
                       workspace=ws)
# remove the downloaded dir after registration if you wish
shutil.rmtree("models")

Registering model inception

## Create and attach remote compute target

Azure Machine Learning service pipelines cannot be run locally, and only run on cloud resources. Remote compute targets are reusable virtual compute environments where you run experiments and work-flows. Run the following code to create a GPU-enabled [`AmlCompute`](https://docs.microsoft.com/python/api/azureml-core/azureml.core.compute.amlcompute.amlcompute?view=azure-ml-py) target, and attach it to your workspace. See the [conceptual article](https://docs.microsoft.com/azure/machine-learning/service/concept-compute-target) for more information on compute targets.

In [0]:
from azureml.core.compute import AmlCompute, ComputeTarget
from azureml.exceptions import ComputeTargetException
compute_name = "gpu-cluster"

# checks to see if compute target already exists in workspace, else create it
try:
    compute_target = ComputeTarget(workspace=ws, name=compute_name)
except ComputeTargetException:
    config = AmlCompute.provisioning_configuration(vm_size="STANDARD_NC6",
                                                   vm_priority="lowpriority", 
                                                   min_nodes=0, 
                                                   max_nodes=1)

    compute_target = ComputeTarget.create(workspace=ws, name=compute_name, provisioning_configuration=config)
    compute_target.wait_for_completion(show_output=True, min_node_count=None, timeout_in_minutes=20)

Creating
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned

## Write a scoring script

To do the scoring, you create a batch scoring script `batch_scoring.py`, and write it to the current directory. The script takes a minibatch of input images, applies the classification model, and outputs the predictions to a results file.

The script `batch_scoring.py` takes the following parameters, which get passed from the `ParallelRunStep` that you create later:

- `--model_name`: the name of the model being used
- `--labels_dir` : the directory path having the `labels.txt` file 

The pipelines infrastructure uses the `ArgumentParser` class to pass parameters into pipeline steps. For example, in the code below the first argument `--model_name` is given the property identifier `model_name`. In the `main()` function, this property is accessed using `Model.get_model_path(args.model_name)`.

The pipeline in this tutorial only has one step and writes the output to a file, but for multi-step pipelines, you also use `ArgumentParser` to define a directory to write output data for input to subsequent steps. See the [notebook](https://github.com/Azure/MachineLearningNotebooks/blob/master/how-to-use-azureml/machine-learning-pipelines/nyc-taxi-data-regression-model-building/nyc-taxi-data-regression-model-building.ipynb) for an example of passing data between multiple pipeline steps using the `ArgumentParser` design pattern.

In [0]:
%%writefile /dbfs/scripts/batch_scoring.py

# Copyright (c) Microsoft. All rights reserved.
# Licensed under the MIT license.

import os
import argparse
import datetime
import time
import tensorflow as tf
from math import ceil
import numpy as np
import shutil
from tensorflow.contrib.slim.python.slim.nets import inception_v3

from azureml.core import Run
from azureml.core.model import Model
from azureml.core.dataset import Dataset

slim = tf.contrib.slim

image_size = 299
num_channel = 3


def get_class_label_dict(labels_dir):
    label = []
    labels_path = os.path.join(labels_dir, 'labels.txt')
    proto_as_ascii_lines = tf.gfile.GFile(labels_path).readlines()
    for l in proto_as_ascii_lines:
        label.append(l.rstrip())
    return label


def init():
    global g_tf_sess, probabilities, label_dict, input_images

    parser = argparse.ArgumentParser(description="Start a tensorflow model serving")
    parser.add_argument('--model_name', dest="model_name", required=True)
    parser.add_argument('--labels_dir', dest="labels_dir", required=True)
    args, _ = parser.parse_known_args()

    label_dict = get_class_label_dict(args.labels_dir)
    classes_num = len(label_dict)

    with slim.arg_scope(inception_v3.inception_v3_arg_scope()):
        input_images = tf.placeholder(tf.float32, [1, image_size, image_size, num_channel])
        logits, _ = inception_v3.inception_v3(input_images,
                                              num_classes=classes_num,
                                              is_training=False)
        probabilities = tf.argmax(logits, 1)

    config = tf.ConfigProto()
    config.gpu_options.allow_growth = True
    g_tf_sess = tf.Session(config=config)
    g_tf_sess.run(tf.global_variables_initializer())
    g_tf_sess.run(tf.local_variables_initializer())

    model_path = Model.get_model_path(args.model_name)
    saver = tf.train.Saver()
    saver.restore(g_tf_sess, model_path)


def file_to_tensor(file_path):
    image_string = tf.read_file(file_path)
    image = tf.image.decode_image(image_string, channels=3)

    image.set_shape([None, None, None])
    image = tf.image.resize_images(image, [image_size, image_size])
    image = tf.divide(tf.subtract(image, [0]), [255])
    image.set_shape([image_size, image_size, num_channel])
    return image


def run(mini_batch):
    result_list = []
    for file_path in mini_batch:
        test_image = file_to_tensor(file_path)
        out = g_tf_sess.run(test_image)
        result = g_tf_sess.run(probabilities, feed_dict={input_images: [out]})
        result_list.append(os.path.basename(file_path) + ": " + label_dict[result[0]])
    return result_list

Overwriting /dbfs/scripts/batch_scoring.py

## Build and run the pipeline

Before running the pipeline, you create an object that defines the python environment and dependencies needed by your script `batch_scoring.py`. The main dependency required is Tensorflow, but you also install `azureml-core` and `azureml-dataset-runtime[fuse]` for background processes from the SDK. Create a `RunConfiguration` object using the dependencies, and also specify Docker and Docker-GPU support.

In [0]:
from azureml.core import Environment
from azureml.core.conda_dependencies import CondaDependencies
from azureml.core.runconfig import DEFAULT_GPU_IMAGE

cd = CondaDependencies.create(pip_packages=["tensorflow-gpu==1.15.2",
                                            "azureml-core", "azureml-dataset-runtime[fuse]"])

env = Environment(name="parallelenv")
env.python.conda_dependencies=cd
env.docker.base_image = DEFAULT_GPU_IMAGE

### Create the configuration to wrap the inference script
Create the pipeline step using the script, environment configuration, and parameters. Specify the compute target you already attached to your workspace as the target of execution of the script. We will use PythonScriptStep to create the pipeline step.

In [0]:
from azureml.pipeline.steps import ParallelRunConfig

parallel_run_config = ParallelRunConfig(
    environment=env,
    entry_script="batch_scoring.py",
    source_directory="/dbfs/scripts",
    output_action="append_row",
    append_row_file_name="parallel_run_step.txt",
    mini_batch_size="20",
    error_threshold=1,
    compute_target=compute_target,
    process_count_per_node=2,
    node_count=1
)

### Create the pipeline step

A pipeline step is an object that encapsulates everything you need for running a pipeline including:

* environment and dependency settings
* the compute resource to run the pipeline on
* input and output data, and any custom parameters
* reference to a script or SDK-logic to run during the step

There are multiple classes that inherit from the parent class [`PipelineStep`](https://docs.microsoft.com/python/api/azureml-pipeline-steps/azureml.pipeline.steps.parallelrunstep?view=azure-ml-py) to assist with building a step using certain frameworks and stacks. In this example, you use the [`ParallelRunStep`](https://docs.microsoft.com/en-us/python/api/azureml-contrib-pipeline-steps/azureml.contrib.pipeline.steps.parallelrunstep?view=azure-ml-py) class to define your step logic using a scoring script. 

An object reference in the `outputs` array becomes available as an **input** for a subsequent pipeline step, for scenarios where there is more than one step.

In [0]:
from azureml.pipeline.steps import ParallelRunStep
from datetime import datetime

parallel_step_name = "batchscoring-" + datetime.now().strftime("%Y%m%d%H%M")

label_config = label_ds.as_named_input("labels_input")

batch_score_step = ParallelRunStep(
    name=parallel_step_name,
    inputs=[input_images.as_named_input("input_images")],
    output=output_dir,
    arguments=["--model_name", "inception",
               "--labels_dir", label_config],
    side_inputs=[label_config],
    parallel_run_config=parallel_run_config,
    allow_reuse=False
)

For a list of all classes for different step types, see the [steps package](https://docs.microsoft.com/python/api/azureml-pipeline-steps/azureml.pipeline.steps?view=azure-ml-py).

### Run the pipeline

Now you run the pipeline. First create a `Pipeline` object with your workspace reference and the pipeline step you created. The `steps` parameter is an array of steps, and in this case there is only one step for batch scoring. To build pipelines with multiple steps, you place the steps in order in this array.

Next use the `Experiment.submit()` function to submit the pipeline for execution. You also specify the custom parameter `param_batch_size`. The `wait_for_completion` function will output logs during the pipeline build process, which allows you to see current progress.

Note: The first pipeline run takes roughly **15 minutes**, as all dependencies must be downloaded, a Docker image is created, and the Python environment is provisioned/created. Running it again takes significantly less time as those resources are reused. However, total run time depends on the workload of your scripts and processes running in each pipeline step.

In [0]:
from azureml.core import Experiment
from azureml.pipeline.core import Pipeline

pipeline = Pipeline(workspace=ws, steps=[batch_score_step])
pipeline_run = Experiment(ws, "batch_scoring").submit(pipeline)

--------------------------------------------------------------------------- 
 ValueError Traceback (most recent call last)
 <command-3981079154799686> in <module> 
 2 from azureml . pipeline . core import Pipeline
 3 
 ----> 4 pipeline = Pipeline ( workspace = ws , steps = [ batch_score_step ] ) 
 5 pipeline_run = Experiment ( ws , "batch_scoring" ) . submit ( pipeline ) 

 /local_disk0/.ephemeral_nfs/envs/pythonEnv-1291aec0-13fd-49c0-a374-30251492cbed/lib/python3.7/site-packages/azureml/core/_experiment_method.py in wrapper (self, *args, **kwargs) 
 95 """
 96 ExperimentSubmitRegistrar . register_submit_function ( self . __class__ , submit_function ) 
 ---> 97 return init_func ( self , * args , ** kwargs ) 
 98 return wrapper
 99 return real_decorator

 /local_disk0/.ephemeral_nfs/envs/pythonEnv-1291aec0-13fd-49c0-a374-30251492cbed/lib/python3.7/site-packages/azureml/pipeline/core/pipeline.py in __init__ (self, workspace, steps, description, default_datastore, default_source_directory, resolve_closure, _workflow_provider, _service_endpoint, **kwargs) 
 175 raise ValueError ( 'parameter %s is not recognized for Pipeline ' % key ) 
 176 self . _enable_email_notification = enable_email_notification
 --> 177 self . _graph = self . _graph_builder . build ( self . _name , steps , finalize = False ) 
 178 
 179 def _set_experiment_name ( self , name ) : 

 /local_disk0/.ephemeral_nfs/envs/pythonEnv-1291aec0-13fd-49c0-a374-30251492cbed/lib/python3.7/site-packages/azureml/pipeline/core/builder.py in build (self, name, steps, finalize, regenerate_outputs) 
 1479 pass 
 1480 
 -> 1481 graph = self . construct ( name , steps ) 
 1482 if finalize : 
 1483 graph . finalize ( regenerate_outputs = regenerate_outputs ) 

 /local_disk0/.ephemeral_nfs/envs/pythonEnv-1291aec0-13fd-49c0-a374-30251492cbed/lib/python3.7/site-packages/azureml/pipeline/core/builder.py in construct (self, name, steps) 
 1501 self . _graph = Graph ( name , self . _context ) 
 1502 self . _nodeStack . append ( [ ] ) 
 -> 1503 self . process_collection ( steps ) 
 1504 for builder in self . _builderStack [ : : - 1 ] : 
 1505 builder . apply_rules ( ) 

 /local_disk0/.ephemeral_nfs/envs/pythonEnv-1291aec0-13fd-49c0-a374-30251492cbed/lib/python3.7/site-packages/azureml/pipeline/core/builder.py in process_collection (self, collection) 
 1537 self . _nodeStack . append ( [ ] ) 
 1538 self . _builderStack . append ( builder ) 
 -> 1539 builder . process_collection ( collection ) 
 1540 added_nodes = self . _nodeStack . pop ( ) 
 1541 self . _nodeStack [ - 1 ] . extend ( added_nodes ) 

 /local_disk0/.ephemeral_nfs/envs/pythonEnv-1291aec0-13fd-49c0-a374-30251492cbed/lib/python3.7/site-packages/azureml/pipeline/core/builder.py in process_collection (self, collection) 
 1828 """
 1829 for item in collection : 
 -> 1830 self . _base_builder . process_collection ( item ) 
 1831 
 1832 def apply_rules ( self ) : 

 /local_disk0/.ephemeral_nfs/envs/pythonEnv-1291aec0-13fd-49c0-a374-30251492cbed/lib/python3.7/site-packages/azureml/pipeline/core/builder.py in process_collection (self, collection) 
 1531 # just a step? 
 1532 if isinstance ( collection , PipelineStep ) : 
 -> 1533 return self . process_step ( collection ) 
 1534 
 1535 # delegate to correct builder 

 /local_disk0/.ephemeral_nfs/envs/pythonEnv-1291aec0-13fd-49c0-a374-30251492cbed/lib/python3.7/site-packages/azureml/pipeline/core/builder.py in process_step (self, step) 
 1575 return self . _step2node [ step ] 
 1576 
 -> 1577 node = step . create_node ( self . _graph , self . _default_datastore , self . _context ) 
 1578 self . assert_node_valid ( step , self . _graph , node ) 
 1579 

 /local_disk0/.ephemeral_nfs/envs/pythonEnv-1291aec0-13fd-49c0-a374-30251492cbed/lib/python3.7/site-packages/azureml/pipeline/steps/parallel_run_step.py in create_node (self, graph, default_datastore, context) 
 170 : rtype : azureml . pipeline . core . graph . Node
 171 """
 --> 172 return super ( ParallelRunStep , self ) . create_node (

In [0]:
# This will output information of the pipeline run, including the link to the details page of portal.
pipeline_run

Experiment,Id,Type,Status,Details Page,Docs Page
batch_scoring,3df13f5a-5048-484e-8a45-a40881189ca3,azureml.PipelineRun,NotStarted,Link to Azure Machine Learning studio,Link to Documentation


In [0]:
# Wait the run for completion and show output log to console
pipeline_run.wait_for_completion(show_output=True)

PipelineRunId: 3df13f5a-5048-484e-8a45-a40881189ca3
Link to Azure Machine Learning Portal: https://ml.azure.com/experiments/batch_scoring/runs/3df13f5a-5048-484e-8a45-a40881189ca3?wsid=/subscriptions/d30d76c0-804c-4f4e-ab7b-509d06a49550/resourcegroups/vlk-workshop/workspaces/vlk-workspace
PipelineRun Status: Running


StepRunId: 70f76633-260a-4c25-967f-ea40e74b5983
Link to Azure Machine Learning Portal: https://ml.azure.com/experiments/batch_scoring/runs/70f76633-260a-4c25-967f-ea40e74b5983?wsid=/subscriptions/d30d76c0-804c-4f4e-ab7b-509d06a49550/resourcegroups/vlk-workshop/workspaces/vlk-workspace
StepRun( batchscoring-202011021034 ) Status: Queued

Streaming azureml-logs/20_image_build_log.txt
=============================================
2020/11/02 10:37:01 Downloading source code...
2020/11/02 10:37:02 Finished downloading source code
2020/11/02 10:37:03 Creating Docker network: acb_default_network, driver: 'bridge'
2020/11/02 10:37:03 Successfully set up Docker network: acb_default_network
2020/11/02 10:37:03 Setting up Docker configuration...
2020/11/02 10:37:04 Successfully set up Docker configuration
2020/11/02 10:37:04 Logging in to registry: f89c90464c5e495db4f6977a78dd3507.azurecr.io
2020/11/02 10:37:05 Successfully logged into f89c90464c5e495db4f6977a78dd3507.azurecr.io
2020/11/02 10:37:05 Executing step ID: acb_step_0. Timeout(sec): 5400, Working directory: '', Network: 'acb_default_network'
2020/11/02 10:37:05 Scanning for dependencies...
2020/11/02 10:37:06 Successfully scanned dependencies
2020/11/02 10:37:06 Launching container with name: acb_step_0
Sending build context to Docker daemon 60.93kB

Step 1/15 : FROM mcr.microsoft.com/azureml/intelmpi2018.3-cuda10.0-cudnn7-ubuntu16.04:20200821.v1@sha256:652cba03ef8fd4937111da0bc9f204a4310105e87b08eea322c1b55850169a49
sha256:652cba03ef8fd4937111da0bc9f204a4310105e87b08eea322c1b55850169a49: Pulling from azureml/intelmpi2018.3-cuda10.0-cudnn7-ubuntu16.04
f7277927d38a: Pulling fs layer
8d3eac894db4: Pulling fs layer
edf72af6d627: Pulling fs layer
3e4f86211d23: Pulling fs layer
d6e9603ff777: Pulling fs layer
5cad422780e2: Pulling fs layer
8130687c8acb: Pulling fs layer
c11e9246d621: Pulling fs layer
0dfae24cbbd9: Pulling fs layer
0bb049a6d391: Pulling fs layer
1aef3da5021b: Pulling fs layer
e5f03d63d02d: Pulling fs layer
22533bcd68c8: Pulling fs layer
a8c7deb5dc88: Pulling fs layer
ef34b1adad4d: Pulling fs layer
87e1451a8ae7: Pulling fs layer
6a5d94939972: Pulling fs layer
f4b1ab4ce7b5: Pulling fs layer
3e4f86211d23: Waiting
d6e9603ff777: Waiting
5cad422780e2: Waiting
8130687c8acb: Waiting
c11e9246d621: Waiting
0dfae24cbbd9: Waiting
0bb049a6d391: Waiting
1aef3da5021b: Waiting
e5f03d63d02d: Waiting
22533bcd68c8: Waiting
a8c7deb5dc88: Waiting
ef34b1adad4d: Waiting
87e1451a8ae7: Waiting
6a5d94939972: Waiting
f4b1ab4ce7b5: Waiting
edf72af6d627: Verifying Checksum
edf72af6d627: Download complete
8d3eac894db4: Verifying Checksum
8d3eac894db4: Download complete
3e4f86211d23: Verifying Checksum
3e4f86211d23: Download complete
f7277927d38a: Verifying Checksum
f7277927d38a: Download complete
StepRun( batchscoring-202011021034 ) Status: Running
8130687c8acb: Verifying Checksum
8130687c8acb: Download complete
d6e9603ff777: Verifying Checksum
d6e9603ff777: Download complete
5cad422780e2: Verifying Checksum
5cad422780e2: Download complete
f7277927d38a: Pull complete
8d3eac894db4: Pull complete
edf72af6d627: Pull complete
3e4f86211d23: Pull complete
d6e9603ff777: Pull complete
5cad422780e2: Pull complete
0dfae24cbbd9: Verifying Checksum
0dfae24cbbd9: Download complete
c11e9246d621: Verifying Checksum
c11e9246d621: Download complete
8130687c8acb: Pull complete
1aef3da5021b: Verifying Checksum
1aef3da5021b: Download complete
e5f03d63d02d: Verifying Checksum
e5f03d63d02d: Download complete
22533bcd68c8: Verifying Checksum
22533bcd68c8: Download complete
a8c7deb5dc88: Verifying Checksum
a8c7deb5dc88: Download complete
ef34b1adad4d: Verifying Checksum
ef34b1adad4d: Download complete
6a5d9

### Download and review output

Run the following code to download the output file created from the `batch_scoring.py` script, then explore the scoring results.

In [0]:
import pandas as pd
import tempfile

batch_run = pipeline_run.find_step_run(batch_score_step.name)[0]
batch_output = batch_run.get_output_data(output_dir.name)

target_dir = tempfile.mkdtemp()
batch_output.download(local_path=target_dir)
result_file = os.path.join(target_dir, batch_output.path_on_datastore, parallel_run_config.append_row_file_name)

df = pd.read_csv(result_file, delimiter=":", header=None)
df.columns = ["Filename", "Prediction"]
print("Prediction has ", df.shape[0], " rows")
df.head(10) 

,Filename,Prediction
0,n02691156_5992_airplane.jpg,rapeseed
1,n02764044_41120_axe.jpg,"""plane, carpenters plane, woodworking plane"""
2,n02766320_8764_baby_bed.jpg,"crib, cot"
3,n02769748_23541_backpack.jpg,"backpack, back pack, knapsack, packsack, ruck..."
4,n02769748_3207_backpack.jpg,"backpack, back pack, knapsack, packsack, ruck..."
5,n02799071_5092_baseball.jpg,baseball
6,n02802426_5133_basketball.jpg,basketball
7,n02815834_4632_beaker.jpg,beaker
8,n02828884_8228_bench.jpg,park bench
9,n02834778_5255_bicycle.jpg,"mountain bike, all-terrain bike, off-roader"


## Publish and run from REST endpoint

Run the following code to publish the pipeline to your workspace. In your workspace in the portal, you can see metadata for the pipeline including run history and durations. You can also run the pipeline manually from the portal.

Additionally, publishing the pipeline enables a REST endpoint to rerun the pipeline from any HTTP library on any platform.

In [0]:
published_pipeline = pipeline_run.publish_pipeline(
    name="Inception_v3_scoring", description="Batch scoring using Inception v3 model", version="1.0")

published_pipeline

Name,Id,Status,Endpoint
Inception_v3_scoring,a7a79c4c-6056-4a29-8720-6dd86969271c,Active,REST Endpoint


To run the pipeline from the REST endpoint, you first need an OAuth2 Bearer-type authentication header. This example uses interactive authentication for illustration purposes, but for most production scenarios requiring automated or headless authentication, use service principle authentication as [described in this notebook](https://aka.ms/pl-restep-auth).

Service principle authentication involves creating an **App Registration** in **Azure Active Directory**, generating a client secret, and then granting your service principal **role access** to your machine learning workspace. You then use the [`ServicePrincipalAuthentication`](https://docs.microsoft.com/python/api/azureml-core/azureml.core.authentication.serviceprincipalauthentication?view=azure-ml-py) class to manage your auth flow. 

Both `InteractiveLoginAuthentication` and `ServicePrincipalAuthentication` inherit from `AbstractAuthentication`, and in both cases you use the `get_authentication_header()` function in the same way to fetch the header.

In [0]:
from azureml.core.authentication import InteractiveLoginAuthentication

interactive_auth = InteractiveLoginAuthentication()
auth_header = interactive_auth.get_authentication_header()

Get the REST url from the `endpoint` property of the published pipeline object. You can also find the REST url in your workspace in the portal. Build an HTTP POST request to the endpoint, specifying your authentication header. Additionally, add a JSON payload object with the experiment name and the batch size parameter. As a reminder, the `process_count_per_node` is passed through to `ParallelRunStep` because you defined it is defined as a `PipelineParameter` object in the step configuration.

Make the request to trigger the run. Access the `Id` key from the response dict to get the value of the run id.

In [0]:
import requests

rest_endpoint = published_pipeline.endpoint
response = requests.post(rest_endpoint, 
                         headers=auth_header, 
                         json={"ExperimentName": "batch_scoring",
                               "ParameterAssignments": {"process_count_per_node": 6}})

In [0]:
try:
    response.raise_for_status()
except Exception:    
    raise Exception("Received bad response from the endpoint: {}\n"
                    "Response Code: {}\n"
                    "Headers: {}\n"
                    "Content: {}".format(rest_endpoint, response.status_code, response.headers, response.content))

run_id = response.json().get('Id')
print('Submitted pipeline run: ', run_id)

Submitted pipeline run: a4390ca7-14c1-4d83-a940-2a63e6c5597c

Use the run id to monitor the status of the new run. This will take another 10-15 min to run and will look similar to the previous pipeline run, so if you don't need to see another pipeline run, you can skip watching the full output.

In [0]:
from azureml.pipeline.core.run import PipelineRun

published_pipeline_run = PipelineRun(ws.experiments["batch_scoring"], run_id)

In [0]:
# Show detail information of the run
published_pipeline_run

Experiment,Id,Type,Status,Details Page,Docs Page
batch_scoring,a4390ca7-14c1-4d83-a940-2a63e6c5597c,azureml.PipelineRun,Running,Link to Azure Machine Learning studio,Link to Documentation


## Clean up resources

Do not complete this section if you plan on running other Azure Machine Learning service tutorials.

### Stop the notebook VM

If you used a cloud notebook server, stop the VM when you are not using it to reduce cost.

1. In your workspace, select **Compute**.
1. Select the **Notebook VMs** tab in the compute page.
1. From the list, select the VM.
1. Select **Stop**.
1. When you're ready to use the server again, select **Start**.

### Delete everything

If you don't plan to use the resources you created, delete them, so you don't incur any charges.

1. In the Azure portal, select **Resource groups** on the far left.
1. From the list, select the resource group you created.
1. Select **Delete resource group**.
1. Enter the resource group name. Then select **Delete**.

You can also keep the resource group but delete a single workspace. Display the workspace properties and select **Delete**.

## Next steps

In this machine learning pipelines tutorial, you did the following tasks:

> * Built a pipeline with environment dependencies to run on a remote GPU compute resource
> * Created a scoring script to run batch predictions with a pre-trained Tensorflow model
> * Published a pipeline and enabled it to be run from a REST endpoint

See the [how-to](https://docs.microsoft.com/azure/machine-learning/service/how-to-create-your-first-pipeline?view=azure-devops) for additional detail on building pipelines with the machine learning SDK.

Out[29]: [FileInfo(path='dbfs:/FileStore/import-stage/', name='import-stage/', size=0)]